In [1]:
from functools import reduce
import csbgnpy.pd.io.sbgnml
import pypint
import multiprocessing
from IPython.display import display
import networkx as nx
from tqdm.notebook import tqdm

This notebook has been executed using the docker image `pauleve/clockcycle:v1`

## Merged map

From [1 - Merge CYCLE and CLOCK](1%20-%20Merge%20CYCLE%20and%20CLOCK.ipynb)

In [2]:
merge_file = "generated/MERGE.sbgnml"
merge = csbgnpy.pd.io.sbgnml.read(merge_file)

In [3]:
clock = csbgnpy.pd.io.sbgnml.read("generated/CLOCK.sbgnml")
cycle = csbgnpy.pd.io.sbgnml.read("generated/CYCLE.sbgnml")
clock_ids = set([e.id for e in clock.entities])# + [p.id for p in clock.processes])
cycle_ids = set([e.id for e in cycle.entities])# + [p.id for p in cycle.processes])
clock_only = clock_ids - cycle_ids
cycle_only = cycle_ids - clock_ids

## Markers

From [2 - Markers.ipynb](2%20-%20Markers.ipynb):

In [4]:
%store -r cycle_markers
%store -r clock_markers

no stored variable or alias cycle_markers
no stored variable or alias clock_markers


## Initial state

From [3 - Initial state](3%20-%20Initial%20state.ipynb)

In [5]:
%store -r inis

To ids:

In [6]:
inis = {code: [e.id for e in ini] for code, ini in inis.items()}

### Conversion to internal identifiers

In [7]:
def get_map(a):
    if a in clock_only:
        return "CLOCK"
    if a in cycle_only:
        return "CYCLE"
    return "MERGE"
def pretty_node(a):
    if a.startswith("epn_"):
        e = merge.get_entity(a, by_id=True)
    else:
        e = merge.get_process(a, by_id=True)
    return "{}::{}".format(get_map(a), str(e))
def get_entity_id(spec):
    e = merge.get_entity(spec, by_string=True)
    if not e:
        raise Exception(f"No entity found for '{spec}'")
    return e.id

In [8]:
an_cycle_markers = dict([(p, [(get_entity_id(m),1) for m in ms]) \
                         for p,ms in cycle_markers.items()])
an_clock_markers = dict([(p, [(get_entity_id(m),1) for m in ms]) \
                         for p,ms in clock_markers.items()])

In [9]:
def pretty_mutations(m):
    return dict([(pretty_node(a), i) for (a,i) in sorted(m.items())])

def has_proc(m):
    for e in m:
        if e.startswith("proc_"):
            return True
    return False

def unpretty_node(a):
    name = "::".join(a.split("::")[1:])
    entity = merge.get_entity(name, by_string=True)
    return entity.id

def unpretty_mutations(m):
    return dict([(unpretty_node(a), i) for (a,i) in m.items()])

In [10]:
def oneshot_mutations_for_cut(an, markers, **kwargs):
    ms = []
    for marker in markers:
        ms += an.oneshot_mutations_for_cut(marker, **kwargs)
    return ms

In [11]:
def check_mutation_for_cut(an, markers, locks):
    for marker in markers:
        wt_reach = an.reachability(marker)
        mu_reach = an.lock(locks).reachability(marker)
        if wt_reach and not mu_reach:
            print("mutation is cutting {}".format(pretty_node(marker[0])))

In [12]:
def list_interesting(ms, interest):
    i2m = {}
    for mi, m in enumerate(ms):
        li = [(a,i) for (a,i) in m.items() if a in interest]
        arity = len(m)
        for (a,i) in li:
            a = pretty_node(a)
            if (a,i) not in i2m:
                i2m[(a,i)] = set()
            i2m[(a,i)].add(arity)
            #if arity not in i2m[(a,i)]:
            #    i2m[(a,i)][arity] = []
            #i2m[(a,i)][arity].append(mi)
    si2m = {}
    for k, v in sorted(i2m.items()):
        si2m[k] = v
    return si2m

In [13]:
ans = {code: pypint.load(merge_file, initial_state=ini)
       for code, ini in tqdm(inis.items())}

In [14]:
def list_mutations(an, markers, scope, maxsize=3):
    ret = []
    for m in markers:
        try:
            ms = an.oneshot_mutations_for_cut([m], quiet=True, maxsize=maxsize)
        except:
            ms = {}
        ret.append((m,frozenset(list_interesting(ms, scope).keys())))
    return tuple(ret)

## CYCLE markers

In [15]:
mutations = {}
phases = an_cycle_markers
for phase in tqdm(phases):
    mutations[phase] = {code: list_mutations(an, an_cycle_markers[phase], clock_ids)
                        for code, an in tqdm(list(ans.items()), phase)}

In [16]:
{phase: len(set(r.values())) for phase, r in mutations.items()}

{'earlyG1': 1, 'lateG1': 1, 'earlyS': 1, 'lateS': 1, 'G2': 3, 'M': 3}

In [17]:
cycle_mutations = mutations
%store cycle_mutations

Stored 'cycle_mutations' (dict)


In [18]:
cycle_mutations

{'earlyG1': {'3121': ((('epn_28', 1), frozenset()),
   (('epn_45', 1), frozenset()),
   (('epn_47', 1), frozenset()),
   (('epn_10', 1),
    frozenset({('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))',
                0)})),
   (('epn_46', 1), frozenset())),
  '3221': ((('epn_28', 1), frozenset()),
   (('epn_45', 1), frozenset()),
   (('epn_47', 1), frozenset()),
   (('epn_10', 1),
    frozenset({('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))',
                0)})),
   (('epn_46', 1), frozenset())),
  '1121': ((('epn_28', 1), frozenset()),
   (('epn_45', 1), frozenset()),
   (('epn_47', 1), frozenset()),
   (('epn_10', 1),
    frozenset({('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))',
                0)})),
   (('epn_46', 1), frozenset())),
  '1221': ((('epn_28', 1), frozenset()),
   (('epn_45', 1), frozenset()),
   (('epn_47', 1), frozenset()),
   (('epn_10', 1),
    frozenset({('MERGE::Macromolecule([][]HSP90#Compartment([]cytoplasm))',
          

In [19]:
def summary(spec):
    (a,i) = spec
    name = a.split("#")[0].split('(')[-1].split(']')[-1]
    mut = "LoF" if i == 0 else "GoF"
    return (name, mut)

In [20]:
imutations = {}
for phase, mut in cycle_mutations.items():
    imutations[phase] = {}
    for code, value in mut.items():
        value = frozenset([summary(m)  for (_,entities) in value for m in entities])
        if value not in imutations[phase]:
            imutations[phase][value] = {code}
        else:
            imutations[phase][value].add(code)
imutations

{'earlyG1': {frozenset({('HSP90', 'LoF')}): {'1111',
   '1112',
   '1121',
   '1122',
   '1211',
   '1212',
   '1221',
   '1222',
   '2111',
   '2112',
   '2121',
   '2122',
   '2211',
   '2212',
   '2221',
   '2222',
   '3111',
   '3112',
   '3121',
   '3122',
   '3211',
   '3212',
   '3221',
   '3222'}},
 'lateG1': {frozenset({('HSP90', 'LoF')}): {'1111',
   '1112',
   '1121',
   '1122',
   '1211',
   '1212',
   '1221',
   '1222',
   '2111',
   '2112',
   '2121',
   '2122',
   '2211',
   '2212',
   '2221',
   '2222',
   '3111',
   '3112',
   '3121',
   '3122',
   '3211',
   '3212',
   '3221',
   '3222'}},
 'earlyS': {frozenset(): {'1111',
   '1112',
   '1121',
   '1122',
   '1211',
   '1212',
   '1221',
   '1222',
   '2111',
   '2112',
   '2121',
   '2122',
   '2211',
   '2212',
   '2221',
   '2222',
   '3111',
   '3112',
   '3121',
   '3122',
   '3211',
   '3212',
   '3221',
   '3222'}},
 'lateS': {frozenset({('HSP90', 'LoF')}): {'1111',
   '1112',
   '1121',
   '1122',
   '1211',
 

## CLOCK markers

In [21]:
mutations = {}
for phase in tqdm(an_clock_markers):
    mutations[phase] = {code: list_mutations(an, an_clock_markers[phase], cycle_ids, maxsize=8)
                        for code, an in tqdm(ans.items(), phase)}

In [22]:
{phase: len(set(r.values())) for phase, r in mutations.items()}

{'RORG': 1, 'SIRT1': 1, 'ARNTL-CLOCK': 1, 'PER-CRY': 1}

In [23]:
clock_mutations = mutations
%store clock_mutations
{phase: set(r.values()) for phase, r in mutations.items()}

Stored 'clock_mutations' (dict)


{'RORG': {((('epn_290', 1), frozenset()),)},
 'SIRT1': {((('epn_308', 1), frozenset()),)},
 'ARNTL-CLOCK': {((('epn_89', 1), frozenset()),)},
 'PER-CRY': {((('epn_75', 1), frozenset()),)}}